In [1]:
import os
import locale
import modal
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset

load_dotenv(override=True)

/workspaces/js-complexity-model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

dataset = load_dataset("aslam-naseer/js-function-complexity-processed")
test = dataset['test']

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


### Fine tuned model

In [10]:
%load_ext autoreload
%autoreload 2

from utils.get_messages import get_messages
from utils.complexity_llm import app, ComplexityLLM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def test_fn(code):
  messages = get_messages(code)

  print("Starting app")
  with app.run():
    llm = ComplexityLLM()
    print("Sending request to LLM")
    score = llm.complexity.remote(messages)

    print("Score:", score)

In [6]:
test_fn(test[5]['code'])

Starting app
Sending request to LLM
Score: 1.2


In [ ]:
# !uv run modal deploy -m utils.complexity_llm

⠦ Creating objects.....
⠏ Creating objects...honPackage:utils: Uploaded 0/3 files
⠹ Creating objects...honPackage:utils: Uploaded 0/3 files
⠴ Creating objects...honPackage:utils: Finalizing index of 3 files
⠇ Creating objects...honPackage:utils: Finalizing index of 3 files
├── 🔨 Created mount PythonPackage:utils
⠙ Creating objects...tyLLM.*...
├── 🔨 Created mount PythonPackage:utils
⠸ Creating objects...
├── 🔨 Created mount PythonPackage:utils
└── 🔨 Created function ComplexityLLM.*.
✓ Created objects.
├── 🔨 Created mount PythonPackage:utils
└── 🔨 Created function ComplexityLLM.*.
✓ App deployed in 2.988s! 🎉

View Deployment: 
https://modal.com/apps/aslam-naseer/main/deployed/complexity-service


In [11]:
Complexity = modal.Cls.from_name("complexity-service", "ComplexityLLM")
complexity_llm = Complexity()

def text_fn_main(code):
    messages = get_messages(code)
    score = complexity_llm.complexity.remote(messages)
    return score

In [13]:
text_fn_main(test[5]['code'])

1.2